In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [4]:
# Convert labels to one-hot encoding
batch_size =32
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(r'C:\Users\ACER\Birds\archive\train',  # Path to the training set
    target_size=(224, 224),    # Adjust the size based on your choice
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 84635 images belonging to 525 classes.


In [5]:
valid_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = valid_datagen.flow_from_directory(r'C:\Users\ACER\Birds\archive\valid',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
)

Found 2625 images belonging to 525 classes.


In [8]:
num_classes = 525  # Change this according to the number of classes in your dataset

In [9]:
from tensorflow.keras import layers, models

# Initialize the Sequential model
model = models.Sequential()

# Add a Convolutional layer with Batch Normalization
model.add(layers.Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Add another Convolutional layer with Batch Normalization
model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (3, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output and add Dense layers
model.add(layers.Flatten())
model.add(layers.Dense(128))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

# Add the output layer with the specified number of classes
model.add(layers.Dense(num_classes, activation='softmax'))

In [10]:
# Define callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

initial_lr = 0.001

def lr_scheduler(epoch, lr):
    if epoch % 5 == 0:
        return lr * 0.9
    return lr

lr_schedule = LearningRateScheduler(lr_scheduler)

In [11]:
# Define callbacks
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

In [12]:
#compile the model
model.compile(optimizer=Adam(learning_rate=initial_lr), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
#Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=40,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[lr_schedule,early_stopping]
)

Epoch 1/40
2644/2644 [==============================] - 1996s 754ms/step - loss: 5.1429 - accuracy: 0.0677 - val_loss: 3.6458 - val_accuracy: 0.2736 - lr: 9.0000e-04
Epoch 2/40
2644/2644 [==============================] - 2175s 822ms/step - loss: 3.5811 - accuracy: 0.2444 - val_loss: 2.3593 - val_accuracy: 0.4676 - lr: 9.0000e-04
Epoch 3/40
2644/2644 [==============================] - 2502s 946ms/step - loss: 2.8654 - accuracy: 0.3641 - val_loss: 1.7783 - val_accuracy: 0.5770 - lr: 9.0000e-04
Epoch 4/40
2644/2644 [==============================] - 2517s 952ms/step - loss: 2.4066 - accuracy: 0.4469 - val_loss: 1.5062 - val_accuracy: 0.6376 - lr: 9.0000e-04
Epoch 5/40
2644/2644 [==============================] - 2532s 957ms/step - loss: 2.0693 - accuracy: 0.5121 - val_loss: 1.4263 - val_accuracy: 0.6688 - lr: 9.0000e-04
Epoch 6/40
2644/2644 [==============================] - 2029s 767ms/step - loss: 1.7669 - accuracy: 0.5733 - val_loss: 1.1820 - val_accuracy: 0.7172 - lr: 8.1000e-04
Epoc

In [15]:
# Evaluate on the test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(r'C:\Users\ACER\Birds\archive\test',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Accuracy: {test_acc}')

Found 2625 images belonging to 525 classes.
82/82 [==============================] - 19s 232ms/step - loss: 0.8457 - accuracy: 0.8041
Test Accuracy: 0.8041158318519592


In [18]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load an example test image
test_image_path = r'C:\Users\ACER\Birds\archive\test\ABBOTTS BABBLER\2.jpg'
img = image.load_img(test_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Rescale pixel values to [0, 1]

In [39]:
# Make predictions
predictions = model.predict(img_array)

1/1 [==============================] - 0s 41ms/step


In [36]:
# predictions = model.predict(img_array)
# class_labels = ["Class_0", "Class_1", ..., "Class_524"]

# # Check the shape of predictions
# print(predictions.shape)

# # Inspect the predictions array
# print(predictions[0])

# # Print the length of class_labels
# print(len(class_labels))

# # Get the class index with the highest probability
# predicted_class_index = np.argmax(predictions[0])

# # Print the predicted class and probability
# print(f"Predicted Class Index: {predicted_class_index}")

# # Ensure the predicted_class_index is within the range of class_labels
# if 0 <= predicted_class_index < len(class_labels):
#     print(f"Predicted Class: {class_labels[predicted_class_index]}")
#     print(f"Probability: {predictions[0][predicted_class_index]:.2%}")
# else:
#     print("Invalid predicted class index.")


1/1 [==============================] - 0s 49ms/step
(1, 525)
[1.75487638e-01 1.06945270e-08 7.50023865e-09 5.71197845e-09
 2.57373131e-07 2.43752811e-06 3.16311300e-12 2.42646820e-07
 9.14293502e-11 6.26281338e-16 2.00423474e-05 2.07417887e-08
 4.88891306e-12 3.40537990e-05 2.78135524e-16 3.10093185e-07
 3.78957044e-09 1.24176580e-08 4.44103847e-14 1.36649492e-08
 1.02272701e-09 9.57485380e-09 4.74606621e-09 2.94044433e-08
 4.50435529e-08 1.38718053e-06 1.58665268e-12 7.91690269e-08
 5.53603386e-05 1.45963375e-09 2.80823064e-04 1.44450155e-06
 3.83754286e-06 6.07022770e-08 8.67098606e-08 1.54301603e-07
 1.74147696e-09 3.04363045e-12 4.48283646e-03 7.43775574e-09
 1.11876872e-07 1.45411235e-07 6.61018174e-09 5.12837162e-10
 6.04161396e-05 9.86264497e-02 1.66093305e-04 1.21866709e-04
 7.18413794e-05 7.43643298e-08 8.84161000e-06 2.47903892e-10
 1.21345858e-08 8.32650192e-14 4.97095876e-08 3.19072626e-08
 2.23446001e-12 4.09254753e-05 1.65172480e-03 6.10881762e-06
 2.15705648e-11 7.907137

In [40]:
print(len(class_labels))

4


In [1]:
class_labels = ["Class_0", "Class_1", ..., "Class_524"]
# Get the class index with the highest probability
predicted_class_index = np.argmax(predictions[0])
# Print the predicted class and probability
print(f"Predicted Class Index: {predicted_class_index}")
print(f"Predicted Class: {class_labels[predicted_class_index]}")
print(f"Probability: {predictions[0][predicted_class_index]:.2%}")

NameError: name 'np' is not defined